# Boston Housing Price Prediction Workbook

Tyler Amos

MACSS 30200

In [9]:
from keras import models
from keras import layers
import pandas as pd
import random
random.seed(1234)
import numpy as np

import matplotlib.pyplot as plt
from keras import regularizers

from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [10]:
# Normalize data
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

# Set up the necessary functions

In [12]:
# k fold cross validation
def kfold_valid(model_func, k = 4, num_epochs = 10):
    num_val_samples = len(train_data) // k
    all_scores = []

    for i in range(k):
        print('processing fold #', i)
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
             train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
             train_targets[(i + 1) * num_val_samples:]],
            axis=0)

        model = model_func()
        model.fit(partial_train_data, partial_train_targets,
                  epochs=num_epochs, batch_size=1, verbose=0)
        val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
        all_scores.append(val_mae)
    return all_scores

# Model 1, straight from the book, k = 4, epochs = 100

In [14]:

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

scores = kfold_valid(build_model, 4, 100)
np.mean(scores)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


2.4076350602773156

# model 2, k = 10, epochs = 100

In [15]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

scores = kfold_valid(build_model, 10, 100)
np.mean(scores)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


2.4471402955055237

# model 3, k = 10, epochs = 100, layer size = 80

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(80, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(80, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

scores = kfold_valid(build_model, 10, 100)
np.mean(scores)

processing fold # 0


# model 4, k = 10, epochs = 100, layer size = 50, additional layer with size 25

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(50, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(25, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

scores = kfold_valid(build_model, 10, 100)
np.mean(scores)

# model 4, k = 10, epochs = 100, layer size = 64, additional layer with size 10

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

scores = kfold_valid(build_model, 10, 100)
np.mean(scores)